# 전력수요 예측 특화 LLM - SFT (Supervised Fine-Tuning)

## 접근법
1. **GPT API**로 전력수요 문서 → Q&A 쌍 생성
2. **SFT (지도학습)**으로 Instruction-tuned 모델 파인튜닝
3. 자기지도학습(DAPT) ❌, 지도학습(SFT) ✅

## 왜 SFT?
- Instruct 모델의 질문-답변 능력 유지
- 전력수요 도메인 지식 학습
- DAPT는 instruction 능력을 깨뜨릴 위험

In [ ]:
# 필요한 라이브러리 설치
# !pip install transformers datasets torch accelerate bitsandbytes peft trl sentencepiece openai

In [1]:
import os
import json
from pathlib import Path
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    BitsAndBytesConfig
)
from datasets import Dataset, load_dataset
from trl import SFTTrainer
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import pandas as pd

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

/root/De-Qwen-SFT/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/De-Qwen-SFT/.venv/lib/python3.13/site-packages/trl/__init__.py:203: UserWarning: TRL currently supports vLLM versions: 0.10.2, 0.11.0, 0.11.1, 0.11.2. You have version 0.13.0 installed. We recommend installing a supported version to avoid compatibility issues.
  if is_vllm_available():


PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA device: NVIDIA A100-SXM4-80GB
CUDA memory: 85.09 GB


## 1. Q&A 데이터셋 생성 (GPT API 사용)

먼저 `generate_qa_dataset.py` 스크립트를 실행하여 Q&A 데이터셋을 생성 -> 생성완료

## 2. 생성된 Q&A 데이터셋 로딩 및 확인

In [2]:
# SFT 데이터셋 로딩
dataset_file = "sft_dataset.jsonl"

if not Path(dataset_file).exists():
    print(f"❌ {dataset_file} 파일이 없습니다.")
    print("먼저 generate_qa_dataset.py를 실행하여 데이터셋을 생성하세요.")
else:
    # JSONL 파일 로딩
    with open(dataset_file, 'r', encoding='utf-8') as f:
        sft_data = [json.loads(line) for line in f]
    
    print(f"✓ 총 {len(sft_data)}개 Q&A 쌍 로딩 완료")
    
    # 샘플 확인
    print("\n=== 샘플 데이터 ===\n")
    for i, sample in enumerate(sft_data[:3], 1):
        print(f"[{i}] 질문: {sample['messages'][0]['content']}")
        print(f"    답변: {sample['messages'][1]['content'][:150]}...\n")

✓ 총 1139개 Q&A 쌍 로딩 완료

=== 샘플 데이터 ===

[1] 질문: 2019년 12월의 최대부하는 얼마인가요?
    답변: 2019년 12월의 최대부하는 8,730만kW로 예측되었습니다....

[2] 질문: What was the maximum load in December 2018?
    답변: The maximum load in December 2018 was 8,622 만kW....

[3] 질문: 2019년 12월의 주별 최대 전력 수요는 어떻게 되나요?
    답변: 2019년 12월의 주별 최대 전력 수요는 1주차 8,240만 kW, 2주차 8,420만 kW, 3주차 8,540만 kW, 4주차 8,730만 kW로 증가하는 추세를 보였습니다....



In [3]:
import json
import os

# 파일 경로 설정
input_file = '/root/De-Qwen-SFT/power_demand_sft_full.jsonl'
output_file = '/root/De-Qwen-SFT/power_demand_sft_full_converted.jsonl'

print(f"변환 시작: {input_file} -> {output_file}")

count = 0
with open(input_file, 'r', encoding='utf-8') as infile, \
     open(output_file, 'w', encoding='utf-8') as outfile:
    
    for line in infile:
        if not line.strip():
            continue
            
        data = json.loads(line)
        
        # User Content 구성: Instruction + Input (Input이 있는 경우)
        user_content = data.get('instruction', '')
        input_text = data.get('input', '')
        if input_text:
            user_content += f"\n\n{input_text}"
            
        # Assistant Content 구성: Output
        assistant_content = data.get('output', '')
        
        # 새로운 포맷 (Messages)으로 변환
        new_data = {
            "messages": [
                {"role": "user", "content": user_content},
                {"role": "assistant", "content": assistant_content}
            ]
        }
        
        outfile.write(json.dumps(new_data, ensure_ascii=False) + '\n')
        count += 1

print(f"변환 완료! 총 {count}개의 데이터가 변환되었습니다.")

변환 시작: /root/De-Qwen-SFT/power_demand_sft_full.jsonl -> /root/De-Qwen-SFT/power_demand_sft_full_converted.jsonl
변환 완료! 총 585개의 데이터가 변환되었습니다.


## split 생성

In [4]:
# 수정 전
# dataset_file = "/root/De-Qwen-SFT/sft_dataset.jsonl"
# dataset = load_dataset('json', data_files=dataset_file, split='train')

# 수정 후: 두 파일 경로를 리스트로 전달
dataset_files = [
    "/root/De-Qwen-SFT/sft_dataset.jsonl",
    "/root/De-Qwen-SFT/power_demand_sft_full_converted.jsonl"  # 변환한 파일
]

# 자동으로 두 파일을 합쳐서 로드합니다
dataset = load_dataset('json', data_files=dataset_files, split='train')

# 이후 코드는 그대로 사용 (합쳐진 전체 데이터에서 10%를 테스트 셋으로 분리)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

print(f"Train set: {len(dataset['train'])}개")
print(f"Test set: {len(dataset['test'])}개")

Generating train split: 1724 examples [00:00, 188694.97 examples/s]

Train set: 1551개
Test set: 173개


## 3. 모델 및 토크나이저 로딩

### 추천 모델:
1. **EEVE-Korean-10.8B**: `yanolja/EEVE-Korean-Instruct-10.8B-v1.0`
2. **Qwen2.5-7B**: `Qwen/Qwen2.5-7B-Instruct`

In [5]:
# 모델 선택
model_name = "/root/models/llama-3-korean-bllossom-8B"  # Qwen/Qwen2.5-7B-Instruct

print(f"Loading model: {model_name}")

Loading model: /root/models/llama-3-korean-bllossom-8B


In [6]:
# 토크나이저 로딩
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True
)

# padding token 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

print(f"✓ Tokenizer loaded")
print(f"  Vocab size: {len(tokenizer)}")
print(f"  Pad token: {tokenizer.pad_token}")

✓ Tokenizer loaded
  Vocab size: 128256
  Pad token: <|end_of_text|>


In [7]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# 1. 경로 수정 (방금 다운로드 받은 폴더 이름으로 변경)
model_path = "/root/models/llama-3-korean-bllossom-8B"

print(f"Loading model from: {model_path}")

# 2. 4bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
)

# 3. 토크나이저 로드 (이게 있어야 글자를 이해합니다)
tokenizer = AutoTokenizer.from_pretrained(
    model_path,
    local_files_only=True,
    trust_remote_code=True
)

# 4. 모델 로딩
model = AutoModelForCausalLM.from_pretrained(
    model_path,
    quantization_config=bnb_config,
    device_map="auto",
    local_files_only=True,
    trust_remote_code=True,
    torch_dtype=torch.float16,
)

print(f"✓ Model loaded successfully")
print(f"  Device: {model.device}")

Loading model from: /root/models/llama-3-korean-bllossom-8B


`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 4/4 [00:17<00:00,  4.48s/it]

✓ Model loaded successfully
  Device: cuda:0


## 4. 학습 전 모델 테스트

In [8]:
print(tokenizer.chat_template)

# 실제 chat template 확인
print("=== Chat Template ===")
print(tokenizer.chat_template)

print("\n=== Special Tokens ===")
print(f"EOS token: {tokenizer.eos_token} -> {tokenizer.eos_token_id}")
print(f"BOS token: {tokenizer.bos_token} -> {tokenizer.bos_token_id}")
print(f"PAD token: {tokenizer.pad_token} -> {tokenizer.pad_token_id}")

# 특수 토큰들이 실제로 존재하는지 확인
print("\n=== Special Token IDs ===")
for token in ["<|begin_of_text|>", "<|start_header_id|>", "<|end_header_id|>", "<|eot_id|>"]:
    token_id = tokenizer.convert_tokens_to_ids(token)
    print(f"{token} -> {token_id}")

{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}{% endif %}
=== Chat Template ===
{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>

'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>

' }}{% endif %}

=== Spe

In [9]:
import torch

def format_llama3_chat(prompt: str) -> str:
    # Llama-3는 Header 뒤에 줄바꿈이 2개(\n\n) 들어가야 합니다.
    return (
        "<|begin_of_text|>"
        "<|start_header_id|>user<|end_header_id|>\n\n"  # 수정: \n -> \n\n
        f"{prompt}<|eot_id|>"
        "<|start_header_id|>assistant<|end_header_id|>\n\n"  # 수정: \n -> \n\n
    )

def test_model(prompt, max_new_tokens=200):
    text = format_llama3_chat(prompt)
    
    # tokenizer와 model이 전역 변수로 있다고 가정
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    # pad_token_id 설정 (Llama 3는 보통 end_of_text 토큰을 pad로 씀)
    # config에 따라 다르지만 보통 128001(<|end_of_text|>) 사용
    pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 128001

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            pad_token_id=pad_token_id, 
            eos_token_id=[128001, 128009],  # <|end_of_text|>, <|eot_id|> 모두 종료로 인식
            repetition_penalty=1.1,
        )

    # 입력 토큰 이후부터 디코딩
    generated = outputs[0][inputs["input_ids"].shape[1]:]
    response = tokenizer.decode(generated, skip_special_tokens=True)
    return response

# --- 테스트 실행 ---
test_questions = [
    "2019년 2월 전력수요 최대부하는 얼마인가요?",
    "전력수요 예측에서 기상 요인의 영향을 설명해주세요.",
    "What is the peak electricity demand in summer?"
]

print("=" * 80)
print("학습 전 모델 응답 테스트")
print("=" * 80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=150)}")
    print("-" * 80)

학습 전 모델 응답 테스트

[1] 질문: 2019년 2월 전력수요 최대부하는 얼마인가요?
답변: 2019년 2월의 전력 수요는 특정한 날짜에 대한 정보가므로, 정확한 값을 제공하기 위해서는 해당 시기의 전력 수요 데이터를 확인해야 합니다. 그러나, 전력 수요는 날씨, 경제 활동, 인구 밀집 등 다양한 요인에 의해 변동되기 때문에, 특정한 날짜의 전력 수요를 예측하기 어렵습니다.

그러나, 전 세계적으로 전력 수요는 일반적으로 연중과 계절에 따라 변동합니다. 예를 들어, 겨울철에는 난방이 필요할 수 있지만, 여름철에는 공급량이 줄어야 할 수 있습니다. 또한, 특정한 지역이나
--------------------------------------------------------------------------------

[2] 질문: 전력수요 예측에서 기상 요인의 영향을 설명해주세요.
답변: 기상 요인(Weather Factor)들이 전력 수요 예측에 미치는 영향은 다음과 같습니다.

1. **기온**: 기온이 높거나 낮을 때, 전력 소비가 달라집니다.
   - 높은 기온: 에어컨 사용 증가로 전력 소비 증가
   - 낮은 기온: 난방 장비 사용 증가로 전력 소비 증가

2. **습도**: 습도가 높을 때, 냉난방 장비의 전력 소비가 증가합니다.
   - 습도 증가: 공기 조화 및 냉난방 장비 사용 증가로 전력 소비 증가

3. **풍속**: 강한 바람
--------------------------------------------------------------------------------

[3] 질문: What is the peak electricity demand in summer?
답변: The peak electricity demand, also known as peak load or peak power demand, varies by region and country. It typically occurs during hot summer after

## 5. LoRA 설정

In [10]:
# LoRA 설정
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)

model.print_trainable_parameters()

trainable params: 13,631,488 || all params: 8,043,892,736 || trainable%: 0.1695


## 6. 데이터 포맷팅 함수

In [11]:
def formatting_func(example):
    return tokenizer.apply_chat_template(
        example["messages"],   # ❗ 그대로 전달
        tokenize=False,
        add_generation_prompt=False,
    )

# 샘플 확인
sample = dataset['train'][0]
print("=== 포맷팅 전 ===")
print(sample['messages'])
print("\n=== 포맷팅 후 ===")
print(formatting_func({'messages': [sample['messages']]})[0][:300])

=== 포맷팅 전 ===
[{'role': 'user', 'content': 'What are the expected precipitation levels for June 2021?'}, {'role': 'assistant', 'content': 'The expected precipitation levels for June 2021 are similar or more than average, with a probability of 40% each.'}]

=== 포맷팅 후 ===
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

What are the expected precipitation levels for June 2021?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The expected precipitation levels for June 2021 are similar or more than average, with a probability of 40% each.<|eot_id|>


## 7. SFT 학습 설정

In [12]:
from transformers import TrainingArguments

# 학습 파라미터
training_args = TrainingArguments(
    output_dir="./power_demand_sft_output_llama3",
    num_train_epochs=3,
    per_device_train_batch_size=2,  # GPU 메모리에 맞게 조정
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    save_total_limit=3,
    warmup_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    
    # ▼▼▼ 수정된 부분 (evaluation_strategy -> eval_strategy) ▼▼▼
    eval_strategy="steps", 
    # ▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲▲
    
    load_best_model_at_end=True,
    report_to="none",  # wandb 사용 시 "wandb"로 변경
)

print("✓ Training arguments configured")

✓ Training arguments configured


In [ ]:
## A6000에 최적화된 모델 함수

from trl import SFTTrainer, SFTConfig

# 1. SFTConfig
sft_config = SFTConfig(
    output_dir="./power_demand_sft_output_llama3",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    save_total_limit=3,
    warmup_steps=10,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",
)


print("✓ SFT Config configured")

# 2. SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,   # ✅ tokenizer는 여기
    formatting_func=formatting_func,
)

print("✓ Trainer created")

In [ ]:
## A100 최적화

from trl import SFTTrainer, SFTConfig

# 1. SFTConfig (A100 80GB 최적화)
sft_config = SFTConfig(
    output_dir="./power_demand_sft_output_llama3",
    num_train_epochs=3,
    
    # A100의 대용량 메모리 활용
    per_device_train_batch_size=16,  # 2 -> 16 (메모리 여유에 따라 32까지도 가능)
    per_device_eval_batch_size=16,   # 2 -> 16
    
    # 배치 사이즈가 커졌으므로 accumulation은 줄여도 됨 (Total Batch = 16 * 1 * 1 = 16)
    gradient_accumulation_steps=1,   # 4 -> 1
    
    learning_rate=2e-4,
    fp16=False,
    bf16=True,      # A100은 bf16 지원 (필수)
    
    # 로깅 및 저장 주기 조정
    save_steps=50,
    eval_steps=50,
    logging_steps=10,
    save_total_limit=3,
    warmup_steps=10,
    
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    eval_strategy="steps",
    load_best_model_at_end=True,
    report_to="none",
    
    # 추가 최적화 옵션
    gradient_checkpointing=True,     # 메모리 절약 (큰 배치 사이즈 가능하게 함)
    dataloader_num_workers=4,        # 데이터 로딩 속도 향상
)


print("✓ SFT Config configured (A100 Optimized)")

# 2. SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,   # ✅ tokenizer는 여기
    formatting_func=formatting_func,
)

print("✓ Trainer created")

✓ SFT Config configured (A100 Optimized)


Truncating eval dataset: 100%|██████████| 173/173 [00:00<00:00, 48565.33 examples/s]

✓ Trainer created


In [14]:
model = model.to(dtype=torch.float16)
model.config.torch_dtype = torch.float16
model.config.use_cache = False

## 8. SFT 학습 실행

In [15]:
print("Starting SFT training...")
print(f"Total training samples: {len(dataset['train'])}")
print(f"Total eval samples: {len(dataset['test'])}")

effective_batch_size = (
    sft_config.per_device_train_batch_size
    * sft_config.gradient_accumulation_steps
)

print(f"Effective batch size: {effective_batch_size}")

trainer.train()

print("\n✓ Training completed!")


The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 128009, 'pad_token_id': 128001}.


Starting SFT training...
Total training samples: 1551
Total eval samples: 173
Effective batch size: 16


Step,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
50,0.582300,0.552288,0.607198,136071.000000,0.850849
100,0.399300,0.391773,0.427721,260711.000000,0.882777
150,0.327900,0.358923,0.369956,395761.000000,0.888805
200,0.301900,0.336040,0.338133,518657.000000,0.893952
250,0.283600,0.322758,0.326286,653650.000000,0.898179



✓ Training completed!


## 9. 모델 저장

In [16]:
# 학습된 모델 저장
output_dir = "./power_demand_sft_model_llama3"

trainer.model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to {output_dir}")

✓ Model saved to ./power_demand_sft_model_llama3


## 10. 학습 후 모델 테스트

In [17]:
# 학습 후 테스트
print("=" * 80)
print("학습 후 모델 응답")
print("=" * 80)

for i, q in enumerate(test_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=150)}")
    print("-" * 80)

학습 후 모델 응답

[1] 질문: 2019년 2월 전력수요 최대부하는 얼마인가요?


답변: 2019년 2월 전력수요 최대부하는 8,760만 kW로 예측되었습니다.
--------------------------------------------------------------------------------

[2] 질문: 전력수요 예측에서 기상 요인의 영향을 설명해주세요.
답변: 2024년 1월 전력 수요는 평년보다 높을 것으로 예상되며, 기온과 강수량이 평년보다 낮고 건조할 확률이 50%입니다.
--------------------------------------------------------------------------------

[3] 질문: What is the peak electricity demand in summer?
답변: The peak electricity demand in summer is projected to be 93,400 MW.
--------------------------------------------------------------------------------


In [18]:
# 추가 테스트 질문
additional_questions = [
    "여름철 전력수요가 높은 이유는?",
    "2023년 최대 전력수요는 언제 발생했나요?",
    "What are the main factors affecting power demand?",
    "2025년 1월의 평균 전력수요는 어떻게 될 것으로 생각하나요?",
    "평년 대비 전력수요 증가율을 어떻게 계산하나요?",
]

print("\n" + "=" * 80)
print("추가 테스트")
print("=" * 80)

for i, q in enumerate(additional_questions, 1):
    print(f"\n[{i}] 질문: {q}")
    print(f"답변: {test_model(q, max_new_tokens=200)}")
    print("-" * 80)


추가 테스트

[1] 질문: 여름철 전력수요가 높은 이유는?
답변: 2019년 8월과 9월은 기온이 높고 강수량이 많을 것으로 예상되며, 이는 전력 수요를 증가시킬 수 있습니다.
--------------------------------------------------------------------------------

[2] 질문: 2023년 최대 전력수요는 언제 발생했나요?
답변: 2023년 최대 전력수요는 8월에 92,700㎿로 예상됩니다.
--------------------------------------------------------------------------------

[3] 질문: What are the main factors affecting power demand?
답변: The main factors affecting power demand include temperature, precipitation, and seasonal changes.
--------------------------------------------------------------------------------

[4] 질문: 2025년 1월의 평균 전력수요는 어떻게 될 것으로 생각하나요?
답변: 2025년 1월의 평균 전력수요는 65,800㎿로 예상됩니다.
--------------------------------------------------------------------------------

[5] 질문: 평년 대비 전력수요 증가율을 어떻게 계산하나요?
답변: 평년 대비 전력수요 증가율은 전력수요의 평균치와 비교하여 계산되며, 이는 수요가 평년보다 얼마나 증가했는지 나타냅니다.
--------------------------------------------------------------------------------


## 11. 성능 평가

In [ ]:
# Test set 샘플로 성능 평가
test_samples = dataset['test'].select(range(min(10, len(dataset['test']))))

print("=" * 80)
print("Test Set 샘플 평가")
print("=" * 80)

for i, sample in enumerate(test_samples, 1):
    question = sample['messages'][0]['content']
    expected = sample['messages'][1]['content']
    predicted = test_model(question, max_new_tokens=200)
    
    print(f"\n[{i}] 질문: {question}")
    print(f"\n정답: {expected}")
    print(f"\n예측: {predicted}")
    print("-" * 80)

Test Set 샘플 평가

[1] 질문: What was the trend in maximum load from 2017 to 2021?

정답: The maximum load showed an increase in 2018, then a decrease in 2019 and 2020, followed by an increase again in 2021.

예측: The maximum load showed a general increasing trend from 2017 to 2020 but decreased slightly in 2021.
--------------------------------------------------------------------------------

[2] 질문: 2022년 6월의 평균 전력수요는 어떻게 되나요?

정답: 2022년 6월의 평균 전력수요는 61,700㎿로 예측되고 있습니다.

예측: 2022년 6월의 평균 전력수요는 61,700㎿로 예측됩니다.
--------------------------------------------------------------------------------

[3] 질문: Describe the weekly electricity demand pattern for September 2020.

이 표는 2020년 9월의 주별 기상 예측을 나타내고 있습니다. 각 주의 기온과 강수량에 대한 예측이 포함되어 있습니다.

1주 (8.31~9.6):
- 덥고 습한 공기의 영향을 받다가 상층 찬 공기의 영향을 일시적으로 받아 기온 변화가 크고, 발달한 저기압과 대기 불안정으로 많은 비가 내릴 때가 있겠습니다.
- 평년 대비 기온과 강수량은 비슷하겠습니다.

2주 (9.7~9.13):
- 상층 찬 공기의 영향을 받겠으나, 낮 동안에는 일사로 인해 더운 날이 있겠습니다.
- 평년 대비 기온은 비슷하고, 강수량은 비슷하거나 많겠습니다.

3주 (9.14~9.20):
- 건조한 공기(이동성 고기압

## 12. 결론

### SFT 접근법의 장점:
- ✅ Instruction-following 능력 유지
- ✅ 전력수요 도메인 지식 학습
- ✅ 질문-답변 형태로 즉시 사용 가능

### 다음 단계:
1. **더 많은 데이터**: GPT API로 전체 125개 파일 처리
2. **하이퍼파라미터 튜닝**: learning rate, epochs 조정
3. **평가 메트릭**: BLEU, ROUGE 등으로 정량 평가
4. **배포**: FastAPI 등으로 서빙

### 비용 고려:
- GPT-4o-mini: 약 $0.15/1M input tokens
- 125개 파일 × 평균 5000 tokens = 625K tokens
- 예상 비용: ~$0.10 (매우 저렴)